In [ ]:
# Imports and environment check
import os
import pandas as pd
import iris
from sentence_transformers import SentenceTransformer
from chatbot_core.orchestrator import Orchestrator
from chatbot_core.rag import RAGEngine
from chatbot_core.agents import FormAgent, ProcessAgent, OrgAgent

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    print('Warning: OPENAI_API_KEY not set. LLM calls will fail until configured.')

print('Imports OK — orchestrator module available')

In [ ]:
# Connect to InterSystems IRIS (same as original notebook)
conn = iris.connect('localhost', 32782, 'DEMO', '_SYSTEM', 'ISCDEMO')
cursor = conn.cursor()
print('Connected to IRIS via iris.connect')

In [ ]:
# Load sample JSON dataset (keeps original notebook behavior)
df = pd.read_json('./data/data.json')
print('Loaded', len(df), 'rows from data/data.json')

In [ ]:
# Build the in-memory RAG and preload templates (mirrors original notebook helpers)
orch = Orchestrator(doc_root='./data/')
# Preload docx files into the RAG store (if available)
docs = orch._list_docx_files()
for f in docs:
    try:
        from docx import Document
        doc = Document(os.path.join(orch.doc_root, f))
        text = orch._doc_to_text(doc)
        orch.add_local_document(f, text)
    except Exception as e:
        print('skip', f, e)

print('Orchestrator initialised — local RAG docs count =', len(orch.rag.docs))

In [ ]:
# Example: add a small document from df into local RAG and query it
if 'content' in df.columns:
    orch.add_local_document('sample-row-0', df.loc[0, 'content'])

print('Local RAG size after adding sample row:', len(orch.rag.docs))
print('Example retrieval:', orch.rag.vector_search('medical history', top_k=3))

In [ ]:
# Interactive example using the orchestrator + agents
class NotebookChatInterface:
    def __init__(self, orchestrator):
        self.orch = orchestrator

    def query(self, text):
        return self.orch.handle_query(text)

bot = NotebookChatInterface(orch)
print('bot ready — call bot.query(
)')